<a href="https://colab.research.google.com/github/WoojinJeonkr/DeepLearning/blob/main/Fully_Sharded_Data_Parallel_(FSDP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 완전히 샤딩된 데이터 병렬 (FSDP)
- 내용 출처: [Pytorch - Get started with fully sharded Data Parallel](https://pytorch.org/tutorials/intermediate/FSDP_tutorial.html?utm_source=whats_new_tutorials&utm_medium=FSDP)

## FSDP 작동 방식
- FSDP: DDP 순위 전반에 걸쳐 모델 매개변수, 옵티마이저 상태 및 기울기를 분할하는 데이터 병렬 처리 유형
- DDP(DistributedDataParallel)에서 각 프로세스/워커는 모델의 복제본을 소유하고 데이터 배치를 처리하며, 마지막으로 all-reduce를 사용하여 다른 작업자에 대한 그라디언트 합산 --> 모델 가중치와 옵티마이저 상태는 모든 작업자에 걸쳐 복제
- FSDP GPU 메모리 풋프린트: 모든 작업자에서 DDP보다 작으며 일부 매우 큰 모델의 훈련을 가능하게 하고 훈련 작업에 더 큰 모델 또는 배치 크기를 맞추는 데 도움이 된다.
- 통신 오버랩은 통신 및 계산 중첩과 같은 내부 최적화로 인해 감소된다.

- 각 부분에서 FSDP가 작동하는 방식
  - 생성자: 샤드 모델 매개변수 및 각 순위는 자체 샤드만 유지
  - foward path
    - all_gather를 실행하여 모든 순위에서 모든 샤드를 수집하여 이 FSDP 단위의 전체 매개변수 복구
    - 순방향 계산을 실행하며 방금 수집한 매개변수 샤드를 버림
  - backward path
    - all_gather를 실행하여 모든 순위에서 모든 샤드를 수집하여 이 FSDP 단위의 전체 매개변수 복구
    - 역방향 계산 실행
    - reduce_scatter를 실행하여 그라디언트 동기화하고 매개변수를 버림

![FSDP workflow](https://github.com/WoojinJeonkr/DeepLearning/blob/main/image/FSDP%20workflow.png?raw=true)